In [1]:
import os
import sys

from google.colab import drive
drive.mount('/content/gdrive/')

path_to_files = '/content/gdrive/My Drive/_made/advanced_ml/hw3/corpora/'
os.listdir(path_to_files)

Mounted at /content/gdrive/


['WarAndPeace.txt', 'AnnaKarenina.txt', 'WarAndPeaceEng.txt']

In [23]:
with open(path_to_files + "AnnaKarenina.txt", "r", encoding="utf8") as data:
    t1 = data.read()

with open(path_to_files + "WarAndPeace.txt", "r", encoding="utf8") as data:
    t2 = data.read()

corpus_for_frequency_count = t1 + t2

### 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:

○ подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно
просто опустить, а вот пробелы лучше оставить);

○ возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3
предложения, иначе вряд ли сработает), зашифруйте их посредством
случайной перестановки символов;

○ расшифруйте их таким частотным методом.


In [3]:
t2[500:1000]

'не фрейлины А.П.Шерер, балы в Москве и Петербурге.\n\n\n\n\n\n* * *\n\n\n\nЛев Николаевич ТолстойЧАСТЬ ПЕРВАЯ\n\nЧАСТЬ BTОРАЯ\n\nЧАСТЬ ТРЕТЬЯ\n\n\n\n\n\n* * *\n\n\n\n\n\nЛев Николаевич Толстой\n\nВОЙНА И МИР\n\nТом 1\n\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\n\n\n\nI\n\n\n— Еh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous préviens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocités de cet Antichri'

In [24]:
import re
from collections import Counter
corpus_for_frequency_count = re.sub(r'[^a-zа-я0-9 ]', ' ', corpus_for_frequency_count.lower())
# corpus_for_frequency_count = re.sub(r'[^а-я ]', ' ', corpus_for_frequency_count.lower())
corpus_for_frequency_count = re.sub(r'\s{2,}', ' ', corpus_for_frequency_count)

In [ ]:
Counter(corpus_for_frequency_count)

In [ ]:
len(corpus_for_frequency_count)

2384774

In [17]:
corpus_for_frequency_count[500:1000]

'ом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель по видимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м достоевский нахо'

In [18]:
import numpy as np
import random

def cipher_text_permutation(text, seed = None, return_mapping = False):
    known_symbols = list(set(text))
    if seed:
        np.random.seed(seed)
    permuted_symbols = np.random.choice(known_symbols, size = len(known_symbols), replace = False)
    mapping = dict(zip(known_symbols, permuted_symbols))
    ciphered_text = "".join(mapping[s] for s in text)
    if return_mapping:
        return ciphered_text, mapping 
    return ciphered_text

In [6]:
for seed in range(3):
    print(cipher_text_permutation('Cаша', seed = seed))

шCаC
Cшаш
шаCа


In [7]:
def get_dechiphering_mapping(ciphered, trained_frequency_mapping):
    ciphered_counts = Counter(ciphered).most_common()
    top_known = trained_frequency_mapping.most_common(len(ciphered_counts))
    mapping = {}
    for index, (symbol, _) in enumerate(top_known):
        mapping[ciphered_counts[index][0]] = symbol
    return mapping

def decipher_text_with_mapping(text, mapping):
    return "".join([mapping[s] for s in text])

In [22]:
test_start_index = 888

for test_size in [100, 1000, 10000, 100000]:
    train_text = corpus_for_frequency_count[:test_start_index] + corpus_for_frequency_count[test_start_index + test_size:]
    test_text = corpus_for_frequency_count[test_start_index:(test_start_index + test_size)]
    trained_frequency_mapping = Counter(train_text)
    ciphered_test_text = cipher_text_permutation(test_text, seed = 8)
    print('init:')
    print(test_text)
    print('-'*50)
    print('ciphered:')
    print(ciphered_test_text)
    print('-'*50)
    d_mapping = get_dechiphering_mapping(ciphered_test_text, trained_frequency_mapping)
    print('deciphered:')
    print(decipher_text_with_mapping(ciphered_test_text, d_mapping))
    print('-'*200)

init:
ас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м дост
--------------------------------------------------
ciphered:
ч лжэлбж чскькйлипркслбпхчяфдсз дл лусжилчлялкяхпбклесплбхкв счяжслапсзлнсплфжтовзлбпвптфпклмлилвп с
--------------------------------------------------
deciphered:
ин тя стниоаьаг реыао секилдуомну н ботр и л алкеса зое скавноилто чеом жое дтпйвм севепдеа ш р вено
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
init:
ас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м достоевский находил в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую оценку из статей и книг на всех языках мира

In [9]:
def decipher_accuracy(init_text, deciphered_text):
    return (np.array(list(init_text)) == np.array(list(deciphered_text))).mean()

def decipher_accuracy_vocab_bound(cipher_mapping, decipher_mapping):
    N = len(cipher_mapping)
    matched = 0
    for s in cipher_mapping:
        if decipher_mapping[cipher_mapping[s]] == s:
            matched += 1
    return matched / N

In [10]:
test_start_index = 888

for test_size in [100, 1000, 10000, 100000]:
    train_text = corpus_for_frequency_count[:test_start_index] + corpus_for_frequency_count[test_start_index + test_size:]
    test_text = corpus_for_frequency_count[test_start_index:(test_start_index + test_size)]
    trained_frequency_mapping = Counter(train_text)
    ciphered_test_text, c_mapping = cipher_text_permutation(test_text, seed = 8, return_mapping = True)
    d_mapping = get_dechiphering_mapping(ciphered_test_text, trained_frequency_mapping)
    print('init:')
    print(test_text)  
    print('-'*50)
    print('deciphered:')  
    deciphered_test_text = decipher_text_with_mapping(ciphered_test_text, d_mapping)
    print(deciphered_test_text)
    print('-'*50)
    print(f'Accuracy overall: {decipher_accuracy(test_text, deciphered_test_text):.2f}')
    print(f'% of correctly deciphered symbols: {decipher_accuracy_vocab_bound(c_mapping, d_mapping):.2f}')
    print('-'*200)

init:
это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что н
--------------------------------------------------
deciphered:
мое наяр талник уое ь дис вг твсиоаыаб пезао телиндкорск с мовп и н анлета уое тлачсоинво жеор йое д
--------------------------------------------------
Accuracy overall: 0.24
% of correctly deciphered symbols: 0.08
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
init:
это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное 2 ф м достоевский находил в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения 3 время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне кар

### 2.
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не
так уж прост: после буквы E, которая действительно выделяется частотой, дальше
он анализировал уже конкретные слова и пытался угадать, какими они могли бы
быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что
давайте просто сделаем следующий логический шаг:

○ подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;

○ проведите тестирование аналогично п.1, но при помощи биграмм



In [11]:
def split_to_ngrams(text, n = 2):
    return [text[i:i+n] for i in range(len(text) - (n - 1))]

In [34]:
def get_deciphering_mapping_bigrams(ciphered, reference_corpus):
    ciphered_bigram_counts = Counter(split_to_ngrams(ciphered, n = 2)).most_common()
    ciphered_single_counts = Counter(ciphered).most_common()
    known_bigram_counts = Counter(split_to_ngrams(reference_corpus, n = 2)).most_common()
    known_single_counts = Counter(reference_corpus).most_common()
    mapping = {}
    unpaired = set([])
    for (s1, s2), _ in ciphered_bigram_counts:
        ###checking if we already one or two symbols in bigram to some
        if s1 in mapping and s2 in mapping:
            continue
        elif s1 in mapping and s2 not in mapping:
            recognised = mapping[s1]
            candidates = [cand_pair for (cand_pair, _) in known_bigram_counts if cand_pair[0] == recognised and cand_pair[1] not in mapping.values()]
            if candidates:
                mapping[s2] = candidates[0][1]
            else:
                unpaired.add(s2)
        elif s1 not in mapping and s2 in mapping:
            recognised = mapping[s2]
            candidates = [cand_pair for (cand_pair, _) in known_bigram_counts if cand_pair[1] == recognised and cand_pair[0] not in mapping.values()]
            if candidates:
                mapping[s1] = candidates[0][0]
            else:
                unpaired.add(s1)
        else:
            candidates = [cand_pair for (cand_pair, _) in known_bigram_counts if cand_pair[0] not in mapping.values() and cand_pair[1] not in mapping.values()]
            mapping[s1] = candidates[0][0]
            mapping[s2] = candidates[0][1]
    unpaired = [s for s, _ in ciphered_single_counts if s in unpaired]
    known_unpaired = [s for s, _ in known_single_counts if s not in mapping.values()]
    for s, sk in zip(unpaired, known_unpaired):
        mapping[s] = sk
    return mapping

In [ ]:
d_mapping

In [41]:
test_start_index = 888

for test_size in [100, 1000, 10000, 100000, 1000000]:
    train_text = corpus_for_frequency_count[:test_start_index] + corpus_for_frequency_count[test_start_index + test_size:]
    test_text = corpus_for_frequency_count[test_start_index:(test_start_index + test_size)]
    ciphered_test_text, c_mapping = cipher_text_permutation(test_text, seed = 8, return_mapping = True)
    print('init:')
    print(test_text[:1000])
    print('-'*50)
    # print('ciphered:')
    # print(ciphered_test_text)
    # print('-'*50)
    d_mapping = get_deciphering_mapping_bigrams(ciphered_test_text, train_text)
    deciphered_test_text = decipher_text_with_mapping(ciphered_test_text, d_mapping)
    print('deciphered:')
    print(deciphered_test_text[:1000])
    print(f'Accuracy overall: {decipher_accuracy(test_text, deciphered_test_text):.2f}')
    print(f'% of correctly deciphered symbols: {decipher_accuracy_vocab_bound(c_mapping, d_mapping):.2f}')
    print('-'*200)

init:
это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что н
--------------------------------------------------
deciphered:
но скьшестьюкиасяо спсливсмустмвиоьзьцсч дьост юиклаоевасвсномчсискськю тьсяо стюьбвоикмосы оесро сл
Accuracy overall: 0.00
% of correctly deciphered symbols: 0.00
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
init:
это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное 2 ф м достоевский находил в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения 3 время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне карениной можно составить целую библиотеку я без колеб

Как и ранее, хоть что-то вменяемое можно получить только на достаточно больших объёмах зашифрованного текста

3. 
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают.
Основная часть задания — в том, как можно их улучшить:

○ предложите метод обучения перестановки символов в этом задании,
основанный на MCMC-сэмплировании, но по-прежнему работающий на
основе статистики биграмм;

○ реализуйте и протестируйте его, убедитесь, что результаты улучшились.


В качестве скоринговой функции для варианта дешифровки с ключом $x$ будем использовать $\prod R(c_1,c_2) ^{F_x(c_1, c_2)}$, где $с_1с_2$ - биграмма в исходном алфавите, $R()$ - её встречаемость в обучающем тексте-референсе, $F_x(c_1, c_2)$ - встречаемость биграммы в тексте после его дешифровки ключом $x$

Также будем брать от неё логарифм, и сравнивать для нового варианта дешифровки со старым после перестановки двух символов, если новый вариант имеет большее значение функции скоринга - принимаем его, иначе бросаем монету с вероятностью орла как отношение ${score\_new} / {score\_old}$ и в случае успеха также принимаем новый вариант.

In [117]:
import math

def decipher_text_with_mapping(text, mapping):
    return "".join([mapping[s] for s in text])

def split_to_ngrams(text, n = 2):
    return [text[i:i+n] for i in range(len(text) - (n - 1))]

def get_cipher_score(ciphered_text, mapping, reference_bigrams_counts):
    
    decrypted_text = decipher_text_with_mapping(ciphered_text, mapping)
    current_bigrams_counts = Counter(split_to_ngrams(decrypted_text, n = 2))
    
    cipher_score = 0
    for bigram, cnt in current_bigrams_counts.items():
        if bigram in reference_bigrams_counts:
            cipher_score += cnt * math.log(reference_bigrams_counts[bigram])
            # cipher_score += reference_bigrams_counts[bigram] * math.log(cnt)
    return cipher_score

def coin_flip(p):
    if np.random.random() < p:
        return True
    else:
        return False

def swap_symbols(mapping):
    i, j = np.random.choice(list(range(len(mapping))), 2, replace = False)
    l = list(mapping.items())
    xi, xj = l[i], l[j]
    l[i] = (xi[0], xj[1])
    l[j] = (xj[0], xi[1])
    return dict(l)

def get_random_start_mapping(text):
    known_symbols = list(set(text))
    permuted_symbols = np.random.choice(known_symbols, len(known_symbols), replace = False)
    mapping = dict(zip(known_symbols, permuted_symbols))
    return mapping


def run_mcmc(ciphered_text, reference_bigrams_counts, n_mcmc_iterations = 1000, init_mapping = None, seed = None):

    if seed:
        np.random.seed(seed)

    if init_mapping:
        mapping = init_mapping
    else:
        mapping = get_random_start_mapping(ciphered_text)

    for _ in range(n_mcmc_iterations):

        current_score = get_cipher_score(ciphered_text, mapping, reference_bigrams_counts)
        proposed_mapping = swap_symbols(mapping)
        proposed_score = get_cipher_score(ciphered_text, proposed_mapping, reference_bigrams_counts)
        if proposed_score > current_score:
            mapping = proposed_mapping
        else:
            p = min(1, math.exp(proposed_score - current_score))
            if coin_flip(p):
                mapping = proposed_mapping

    final_score = get_cipher_score(ciphered_text, mapping, reference_bigrams_counts)
    return mapping, final_score

In [66]:
from tqdm.notebook import tqdm

In [118]:
for test_size in [100, 1000, 10000, 100000]:
    train_text = corpus_for_frequency_count[:test_start_index] + corpus_for_frequency_count[test_start_index + test_size:]
    test_text = corpus_for_frequency_count[test_start_index:(test_start_index + test_size)]
    reference_bigrams_counts = Counter(split_to_ngrams(train_text, n = 2))
    ciphered_test_text, c_mapping = cipher_text_permutation(test_text, seed = 8, return_mapping = True)
    best_score = -np.inf
    d_mapping = {}
    for seed in tqdm(range(10)):
        mapping, final_score = run_mcmc(ciphered_test_text, reference_bigrams_counts, n_mcmc_iterations = 1000, seed = seed)
        if final_score > best_score:
            best_score = final_score
            d_mapping = mapping
    print('init:')
    print(test_text[:1000])
    print('-'*50)
    # print('ciphered:')
    # print(ciphered_test_text)
    # print('-'*50)
    deciphered_test_text = decipher_text_with_mapping(ciphered_test_text, d_mapping)
    print('deciphered:')
    print(deciphered_test_text[:1000])
    print(f'Accuracy overall: {decipher_accuracy(test_text, deciphered_test_text):.2f}')
    print(f'% of correctly deciphered symbols: {decipher_accuracy_vocab_bound(c_mapping, d_mapping):.2f}')
    print('-'*200)


init:
ас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м дост
--------------------------------------------------
deciphered:
се ах паестижий кофит порсвднтьен е чтак с в ивропи уто прилетсват моть это дазяль полоздои б к лоет
Accuracy overall: 0.54
% of correctly deciphered symbols: 0.31
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



init:
ас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м достоевский находил в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне карениной можно составить целую библиотеку я без колебаний назвал анну каренину величайшим социальным романом во всей мировой литературе писал томас манн значение романа толстого состоит не в эстетической ценности отдельных картин а в художественной завершенности целого войну и мир толстой называл книгой о прошлом в начале года он просил редактора журнала русский вестник м н каткова в оглавлении и даже в объявлении не называть его сочинение романом для меня это очень важно и потому очень прошу вас об этом толстой мог бы обосновать свое определение жанра книга ссылкой на гегеля которого он внимательно перечит


init:
ас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м достоевский находил в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне карениной можно составить целую библиотеку я без колебаний назвал анну каренину величайшим социальным романом во всей мировой литературе писал томас манн значение романа толстого состоит не в эстетической ценности отдельных картин а в художественной завершенности целого войну и мир толстой называл книгой о прошлом в начале года он просил редактора журнала русский вестник м н каткова в оглавлении и даже в объявлении не называть его сочинение романом для меня это очень важно и потому очень прошу вас об этом толстой мог бы обосновать свое определение жанра книга ссылкой на гегеля которого он внимательно перечит

KeyboardInterrupt: ignored

4. Расшифруйте сообщение:

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸
↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊
↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒
↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←
⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨ
ႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხს
დდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ


In [105]:
ciphered = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"


In [115]:
import re
from collections import Counter

with open(path_to_files + "AnnaKarenina.txt", "r", encoding="utf8") as data:
    t1 = data.read()

with open(path_to_files + "WarAndPeace.txt", "r", encoding="utf8") as data:
    t2 = data.read()

corpus_for_frequency_count = t1 + t2

# corpus_for_frequency_count = re.sub(r'[^a-zа-я0-9 ]', ' ', corpus_for_frequency_count.lower())
corpus_for_frequency_count = re.sub(r'[^а-я ]', ' ', corpus_for_frequency_count.lower())
corpus_for_frequency_count = re.sub(r'\s{2,}', ' ', corpus_for_frequency_count)

reference_bigrams_counts = Counter(split_to_ngrams(corpus_for_frequency_count, n = 2))

In [120]:
best_score = -np.inf
d_mapping = {}

for seed in tqdm(range(800, 850)):
    np.random.seed(seed)
    init_mapping = dict(zip(list(set(ciphered)), np.random.choice([chr(i) for i in range(ord('а'), ord('я') + 1)] + [' '], size = 33, replace = False)))
    mapping, final_score = run_mcmc(ciphered, reference_bigrams_counts, init_mapping = init_mapping, n_mcmc_iterations = 50000, seed = seed)
    if final_score > best_score:
        best_score = final_score
        d_mapping = mapping
    deciphered = decipher_text_with_mapping(ciphered, mapping)
    print('deciphered:')
    print(deciphered)
    print(final_score)
    print('-'*200)

deciphered:
ивлаезнезаматиекодяслькныеалаегочтаекодяслькныетирвтежеэтохоевооюбикашеротодныелихроегдочатстьевродииезвихоезнезвиевмилслаегдсзалькоеаеголжчатиеясрваяслькныеюсллейсеговлимкииечитзидтоиейсмскаиерждвсецотшерокичкоешекачихоекиеоюибсф
1844.7380193520505
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
deciphered:
ноер пя прдрин алсувемаяй рер блчир алсувемаяй ингои ю филыл олльщнарх глилсяй еныгл бслчривим оглснн поныл пя пон одневер бсвпремал р блеючрин увгорувемаяй ьвее цв блоенданн чнипнсилн цвдварн гюсов элих гланчал х арчныл ан льнщвъ
1883.5032220084038
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
deciphered:
тсянодшоднынитов углязвшконяной ринов углязвшкоитюсиопощи ь ос

KeyboardInterrupt: ignored

In [126]:
for seed in tqdm(range(814, 815)):
    np.random.seed(seed)
    init_mapping = dict(zip(list(set(ciphered)), np.random.choice([chr(i) for i in range(ord('а'), ord('я') + 1)] + [' '], size = 33, replace = False)))
    mapping, final_score = run_mcmc(ciphered, reference_bigrams_counts, init_mapping = init_mapping, n_mcmc_iterations = 30000, seed = seed)
    if final_score > best_score:
        best_score = final_score
        d_mapping = mapping
    deciphered = decipher_text_with_mapping(ciphered, mapping)
    print('deciphered:')
    print(deciphered)
    print(final_score)
    print('-'*200)

deciphered:
если дя дишите норжальняй или почти норжальняй текст у этого сообщению которяй легко прочитать скорее дсего дя дсе сшелали прадильно и получите жаксижальняй балл ха послешнее четдертое хашание курса фотю конечно ю ничего не обещаъ
2033.1562098553218
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [129]:
for seed in tqdm(range(8887, 8888)):
    np.random.seed(seed)
    init_mapping = dict(zip(list(set(ciphered)), np.random.choice([chr(i) for i in range(ord('а'), ord('я') + 1)] + [' '], size = 33, replace = False)))
    mapping, final_score = run_mcmc(ciphered, reference_bigrams_counts, init_mapping = init_mapping, n_mcmc_iterations = 30000, seed = seed)
    if final_score > best_score:
        best_score = final_score
        d_mapping = mapping
    deciphered = decipher_text_with_mapping(ciphered, mapping)
    print('deciphered:')
    print(deciphered)
    print(final_score)
    print('-'*200)

deciphered:
если вы вижите норшальный или почти норшальный текст у этомо сообщения который лемко прочитать скорее всемо вы все сжелали правильно и получите шаксишальный балл за послежнее четвертое зажание курса фотя конечно я ничемо не обещаъ
2071.9526071450164
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

